In [1]:
import json
import nltk
from nltk.tokenize import RegexpTokenizer
import string
import math
import os
import time
from collections import namedtuple
import cPickle as pickle
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import random

from multiprocessing.dummy import Pool as ThreadPool
import itertools
from collections import defaultdict

from sklearn.metrics import coverage_error
import sklearn.metrics
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.preprocessing import MultiLabelBinarizer

from gensim.models.doc2vec import Doc2Vec, LabeledSentence

import logging
from logging import info
from functools import partial

from thesis.utils.metrics import *


from keras.layers import Input, Dense, Dropout
from keras.models import Model
from sklearn.model_selection import ParameterSampler
import keras

Using gpu device 0: Tesla K40m (CNMeM is disabled, cuDNN 5105)
/home/stud/shalaby/.virtualenv/thesis-env/local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [3]:
IS_SAMPLE = False

In [4]:
SVM_SEED = 1234
DOC2VEC_SEED = 1234
WORD2VEC_SEED = 1234

In [5]:
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"
MIN_WORD_COUNT = 100
MIN_SIZE = 0
NUM_CORES = 14

In [6]:
GLOBAL_VARS = namedtuple('GLOBAL_VARS', ['MODEL_NAME', 'DOC2VEC_MODEL_NAME', 'DOC2VEC_MODEL', 
                                         'SVM_MODEL_NAME', 'NN_MODEL_NAME'])

In [7]:
VOCAB_MODEL = "vocab_model"
MODEL_PREFIX = "model"
VALIDATION_MATRIX = "validation_matrix.pkl"
TEST_MATRIX = "test_matrix.pkl"
METRICS = "metrics.pkl"
CLASSIFIER = "classifier.pkl"

In [8]:
NN_PARAMETER_SEARCH_PREFIX = "{}_batch_{}_nn_parameter_searches.pkl"

In [9]:
#training_file = "/home/local/shalaby/docs_output_sample_100.json"

root_location = "/mnt/data2/shalaby/"
exports_location = root_location + "exported_data/"

doc2vec_model_save_location = os.path.join(root_location, "parameter_search_doc2vec_models_new", "full")
nn_parameter_search_location = os.path.join(root_location, "nn_parameter_search")
if not os.path.exists(doc2vec_model_save_location):
    os.makedirs(doc2vec_model_save_location)
if not os.path.exists(os.path.join(doc2vec_model_save_location, VOCAB_MODEL)):
    os.makedirs(os.path.join(doc2vec_model_save_location, VOCAB_MODEL))

training_file = root_location + "docs_output.json"

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
classification_index_file = exports_location + "classification_index.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
valid_classes_file = exports_location + "valid_classes.pkl"
valid_subclasses_file = exports_location + "valid_subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"
test_docs_list_file = exports_location + "test_docs_list.pkl"

preprocessed_location = root_location + "preprocessed_data/"

training_preprocessed_files_prefix = preprocessed_location + "training_docs_merged_data_preprocessed-"
training_preprocessed_docids_files_prefix = preprocessed_location + "training_docs_merged_docids_preprocessed-"
validation_preprocessed_files_prefix = preprocessed_location + "validation_docs_merged_data_preprocessed-"
validation_preprocessed_docids_files_prefix = preprocessed_location + "validation_docs_merged_docids_preprocessed-"
test_preprocessed_files_prefix = preprocessed_location + "test_docs_merged_data_preprocessed-"
test_preprocessed_docids_files_prefix = preprocessed_location + "test_docs_merged_docids_preprocessed-"

word2vec_questions_file = result = root_location + 'tensorflow/word2vec/questions-words.txt'

In [10]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
valid_classes = pickle.load(open(valid_classes_file))
valid_subclasses = pickle.load(open(valid_subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))
# classifications_index = pickle.load(open(classification_index_file))
test_docs_list = pickle.load(open(test_docs_list_file))

CPU times: user 16.6 s, sys: 812 ms, total: 17.4 s
Wall time: 17.5 s


In [11]:
len(training_docs_list)

1286325

In [12]:
len(validation_docs_list)

321473

In [13]:
def ensure_disk_location_exists(location):
    if not os.path.exists(location):
        os.makedirs(location)

In [14]:
def get_docs_with_inference(doc2vec_model, doc_classification_map, classifications,
                                           docs_list, file_to_write, preprocessed_files_prefix,
                                           preprocessed_docids_files_prefix):
    """
    Use the trained doc2vec model to get the paragraph vector representations of the validation or test documents
    """

    def infer_one_doc(doc_tuple):
        # doc2vec_model.random = np.random.RandomState(DOC2VEC_SEED)
        doc_id, doc_tokens = doc_tuple
        rep = doc2vec_model.infer_vector(doc_tokens)
        return (doc_id, rep)


    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)):
        info("===== Loading inference vectors")
        inference_labels = []
        inference_vectors_matrix = pickle.load(open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write)))
        info("Loaded inference vectors matrix")
        for i, doc_id in enumerate(docs_list):
            curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
            inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
            if i % 100000 == 0:
                info("Finished {} in validation loading".format(i))
        inference_labels = np.array(inference_labels, dtype=np.int8)
    else:
        inference_documents_reps = {}
        inference_vectors = []
        inference_labels = []
        info("===== Getting vectors with inference")


        # Multi-threaded inference
        inference_docs_iterator = DocumentBatchGenerator(preprocessed_files_prefix,
                                                          preprocessed_docids_files_prefix, batch_size=None)
        generator_func = inference_docs_iterator.__iter__()
        pool = ThreadPool(NUM_CORES)
        # map consumes the whole iterator on the spot, so we have to use itertools.islice to fake mini-batching
        mini_batch_size = 1000
        while True:
            threaded_reps_partial = pool.map(infer_one_doc, itertools.islice(generator_func, mini_batch_size))
            info("Finished: {}".format(str(inference_docs_iterator.curr_index)))
            if threaded_reps_partial:
                # threaded_reps.extend(threaded_reps_partial)
                inference_documents_reps.update(threaded_reps_partial)
            else:
                break

        # create matrix for the inferred vectors
        for doc_id in docs_list:
            inference_vectors.append(inference_documents_reps[doc_id])
            curr_doc_labels = set(doc_classification_map[doc_id]) & classifications_set
            inference_labels.append(one_hot_encoder.get_label_vector(curr_doc_labels))
        inference_vectors_matrix = np.array(inference_vectors)
        inference_labels = np.array(inference_labels, dtype=np.int8)
        pickle.dump(inference_vectors_matrix,
                    open(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, file_to_write), 'w'))

    return inference_vectors_matrix, inference_labels

In [15]:
class OneHotEncoder():
    
    def __init__(self, classifications):
        self.classifications = classifications
        self.one_hot_indices = {}

        # convert character classifications to bit vectors
        for i, clssf in enumerate(classifications):
            bits = [0] * len(classifications)
            bits[i] = 1
            self.one_hot_indices[clssf] = i
    
    def get_label_vector(self, labels):
        """
        classes: array of string with the classes assigned to the instance
        """
        output_vector = [0] * len(self.classifications)
        for label in labels:
            index = self.one_hot_indices[label]
            output_vector[index] = 1
            
        return output_vector

def get_training_data(doc2vec_model, classifications):
    one_hot_encoder = OneHotEncoder(classifications)
    classifications_set = set(classifications)
    training_data = []
    training_labels_mat = np.zeros((len(training_docs_list), len(classifications)), dtype=np.int8)
#     training_data_mat = np.zeros((len(training_docs_list), DOC2VEC_SIZE), dtype=np.float32)
    for i,doc_id in enumerate(training_docs_list):
        # converting from memmap to a normal array
#         normal_array = []
#         normal_array[:] = doc2vec_model.docvecs[doc_id][:]
        normal_array = doc2vec_model.docvecs[doc_id]
        training_data.append(normal_array)
#         eligible_classifications = [clssf for clssf in doc_classification_map[doc_id] if clssf in classifications]
        eligible_classifications = set(doc_classification_map[doc_id]) & classifications_set
        training_labels_mat[i][:] = one_hot_encoder.get_label_vector(eligible_classifications)
        if i % 100000 == 0:
            info("Finished {} in training".format(i))
    info("doing matrix creation")
    training_data_mat = np.array(training_data)
#     training_labels_mat = np.array(training_labels, dtype=np.int8)
    del training_data
    return training_data_mat, training_labels_mat

In [16]:
class DocumentBatchGenerator(object):
    def __init__(self, filename_prefix, filename_docids_prefix, batch_size=10000 ):
        """
        batch_size cant be > 10,000 due to a limitation in doc2vec training, 
        None means no batching (only use for inference)
        """
        assert batch_size <= 10000 or batch_size is None
        self.filename_prefix = filename_prefix
        self.filename_docids_prefix = filename_docids_prefix
        self.curr_lines = []
        self.curr_docids = []
        self.batch_size = batch_size
        self.curr_index = 0
        self.batch_end = -1
    def load_new_batch_in_memory(self):
        del self.curr_lines, self.curr_docids
        self.curr_lines, self.docids = [], []
        info("Loading new batch for index: {}".format(self.curr_index) )
        try:
            with open(self.filename_prefix + str(self.curr_index)) as preproc_file:
                for line in preproc_file:
                    self.curr_lines.append(line.split(" "))
#                     if i % 1000 == 0:
#                         print i
            self.curr_docids = pickle.load(open(self.filename_docids_prefix + str(self.curr_index), "r"))
            self.batch_end = self.curr_index + len(self.curr_lines) -1 
            info("Finished loading new batch")
        except IOError:
            info("No more batches to load, exiting at index: {}".format(self.curr_index))
            raise StopIteration()
    def __iter__(self):
        while True:
            if self.curr_index > self.batch_end:
                self.load_new_batch_in_memory()
            for (doc_id, tokens) in zip(self.curr_docids, self.curr_lines):
                if self.batch_size is not None:
                    curr_batch_iter = 0
                    # divide the document to batches according to the batch size
                    while curr_batch_iter < len(tokens):
                        yield LabeledSentence(words=tokens[curr_batch_iter: curr_batch_iter + self.batch_size], tags=[doc_id])
                        curr_batch_iter += self.batch_size
                else:
                    yield doc_id, tokens
                self.curr_index += 1

## NN Specific Functions

In [17]:
def create_keras_nn_model(input_size, output_size, 
                          first_hidden_layer_size, first_hidden_layer_activation, 
                          second_hidden_layer_size, second_hidden_layer_activation, 
                          input_dropout_do, hidden_dropout_do):
    
    doc_input = Input(shape=(DOC2VEC_SIZE,), name='doc_input')
    if input_dropout_do:
        hidden = Dropout(0.7)(doc_input)
    hidden = Dense(first_hidden_layer_size, activation=first_hidden_layer_activation, 
                   name='hidden_layer_{}'.format(first_hidden_layer_activation))(doc_input if not input_dropout_do else hidden)
    if hidden_dropout_do:
        hidden = Dropout(0.5)(hidden)
    if second_hidden_layer_size is not None:
        hidden = Dense(second_hidden_layer_size, activation=second_hidden_layer_activation, 
                       name='hidden_layer2_{}'.format(second_hidden_layer_activation))(hidden)
    softmax_output = Dense(output_size, activation='sigmoid', name='softmax_output')(hidden)

    model = Model(input=doc_input, output=softmax_output)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')
    
    return model

In [18]:
get_binary_0_5 = lambda x: 1 if x > 0.5 else 0
get_binary_0_5 = np.vectorize(get_binary_0_5)

## Actual Training and Validation Loop

In [25]:
early_stopper_deltas = {
    'sections': 0.0001,
    'classes': 0.00001,
    'subclasses': 0.00001
}
early_stopper_patience = {
    'sections': 5,
    'classes': 10,
    'subclasses': 10
}
epochs_before_validation = {
    'sections': 10,
    'classes': 50,
    'subclasses': 50
}

In [32]:
doc2vec_methods_dict = {
#     'doc2vec_size_50_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None' : 7,
#     'doc2vec_size_50_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None' : 8,
#     'doc2vec_size_50_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_100_w_2_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 9,
#     'doc2vec_size_100_w_5_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 18,
#     'doc2vec_size_100_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 10,
#     'doc2vec_size_100_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 4,
#     'doc2vec_size_100_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 7,
    'doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_200_w_2_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 7,
    'doc2vec_size_200_w_4_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 6,
#     'doc2vec_size_200_w_4_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
    'doc2vec_size_200_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 14,
#     'doc2vec_size_200_w_8_type_dm_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 6,
#     'doc2vec_size_200_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_500_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_500_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 8,
#     'doc2vec_size_1000_w_8_type_pv-dbow_concat_1_mean_0_trainwords_0_hs_0_neg_10_vocabsize_None': 6,
#     'doc2vec_size_1000_w_8_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None': 5
}

In [33]:
classifications = valid_subclasses
classifications_type = 'subclasses'

In [34]:
class MetricsCallback(keras.callbacks.Callback):
    
    EPOCHS_BEFORE_VALIDATION = epochs_before_validation[classifications_type]
    
    def on_train_begin(self, logs={}):
        self.epoch_index = 0
        self.val_loss_reductions = 0
        self.metrics_dict = {}
        self.best_val_loss = np.iinfo(np.int32).max
        self.best_weights = None
        self.best_validation_metrics = None
    def on_epoch_end(self, epoch, logs={}):
        self.epoch_index += 1
        if logs['val_loss'] < self.best_val_loss:
            self.val_loss_reductions += 1
            self.best_val_loss = logs['val_loss']
            self.best_weights = self.model.get_weights()
            print '\r    \r' # to remove the previous line of verbose output of model fit
            time.sleep(0.2)
            info('Found lower val loss for epoch {} => {}'.format(self.epoch_index, round(logs['val_loss'], 5)))
            if self.val_loss_reductions % MetricsCallback.EPOCHS_BEFORE_VALIDATION == 0:
                
                info('Validation Loss Reduced {} times'.format(self.val_loss_reductions))
                info('Evaluating on Validation Data')
                yvp = self.model.predict(Xv)
                yvp_binary = get_binary_0_5(yvp)
                info('Generating Validation Metrics')
                validation_metrics = get_metrics(yv, yvp, yvp_binary)
                print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
                    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
                    validation_metrics['f1_micro'], validation_metrics['f1_macro'])
                self.metrics_dict[self.epoch_index] = validation_metrics
#                 self.best_validation_metrics = validation_metrics

In [35]:
NN_OUTPUT_NEURONS = len(classifications)

EARLY_STOPPER_MIN_DELTA = early_stopper_deltas[classifications_type]
EARLY_STOPPER_PATIENCE = early_stopper_patience[classifications_type]

NN_MAX_EPOCHS = 100
NN_RANDOM_SEARCH_BUDGET = 20
NN_PARAM_SAMPLE_SEED = 1234

NN_BATCH_SIZE = 4096

MODEL_VERBOSITY = 1

to_skip = []

load_existing_results = True
save_results = True


first_hidden_layer_sizes = [100,200,500]
# first_hidden_layer_sizes = [1000,2000]
# second_hidden_layer_sizes = [1000,2000,3000,4000]
second_hidden_layer_sizes = [None,500,1000,2000]
first_hidden_layer_activations = ['relu','sigmoid', 'tanh']
second_hidden_layer_activations = ['relu','sigmoid', 'tanh']
# first_hidden_layer_activations = ['relu']
# second_hidden_layer_activations = ['relu']
# input_dropout_options = [False, True]
# hidden_dropout_options = [False, True]
input_dropout_options = [False]
hidden_dropout_options = [False, True]




# Uncomment for Specific Configuration
NN_RANDOM_SEARCH_BUDGET = 1
first_hidden_layer_sizes = [500]
second_hidden_layer_sizes = [2000]
first_hidden_layer_activations = ['tanh']
second_hidden_layer_activations = ['relu']
input_dropout_options = [False]
hidden_dropout_options = [True]

In [ ]:
for (doc2vec_method_name, epoch) in sorted(doc2vec_methods_dict.items(), key=lambda x: x[0]):
    print '********* {}->{}'.format(doc2vec_method_name, epoch)

    TRAINING_METRICS_FILENAME = '{}_training_metrics.pkl'.format(classifications_type)
    VALIDATION_METRICS_FILENAME= '{}_validation_metrics.pkl'.format(classifications_type)
    TEST_METRICS_FILENAME = '{}_test_metrics.pkl'.format(classifications_type)

    placeholder_model_name = doc2vec_method_name
    placeholder_model_name = os.path.join(placeholder_model_name, "epoch_{}")
    GLOBAL_VARS.DOC2VEC_MODEL_NAME = doc2vec_method_name
    
    GLOBAL_VARS.MODEL_NAME = placeholder_model_name.format(epoch)
    
    # if we have the model, just load it, otherwise train the previous model
    if os.path.exists(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX)):
        doc2vec_model = Doc2Vec.load(os.path.join(doc2vec_model_save_location, GLOBAL_VARS.MODEL_NAME, MODEL_PREFIX))
        GLOBAL_VARS.DOC2VEC_MODEL = doc2vec_model
    else:
        info("Couldnt find the doc2vec model with epoch {}".format(epoch))
        raise Exception()


    info('Getting training Data')
    X, y = get_training_data(doc2vec_model, classifications)

    info('Getting Validation Embeddings')
    Xv, yv = get_docs_with_inference(doc2vec_model, doc_classification_map, classifications, 
                                     validation_docs_list, VALIDATION_MATRIX, 
                                     validation_preprocessed_files_prefix, 
                                     validation_preprocessed_docids_files_prefix)
    # create nn parameter search directory
    if not os.path.exists(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME)):
        os.makedirs(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME))
        
    DOC2VEC_SIZE = X.shape[1]
        
    param_sampler = ParameterSampler({
        'first_hidden_layer_size':first_hidden_layer_sizes,
        'first_hidden_layer_activation':first_hidden_layer_activations,
        'second_hidden_layer_size':second_hidden_layer_sizes,
        'second_hidden_layer_activation':second_hidden_layer_activations,
        'input_dropout':input_dropout_options,
        'hidden_dropout':hidden_dropout_options
    }, n_iter=NN_RANDOM_SEARCH_BUDGET, random_state=NN_PARAM_SAMPLE_SEED)
    
    param_results_dict = {}
    if load_existing_results:
        param_results_path = os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE)))
        if os.path.exists(param_results_path):
            param_results_dict = pickle.load(open(param_results_path))
        else:
            info('No Previous results exist in {}'.format(param_results_path))
    
    for parameters in param_sampler:
        start_time = time.time()
        first_hidden_layer_size = parameters['first_hidden_layer_size']
        first_hidden_layer_activation = parameters['first_hidden_layer_activation']
        second_hidden_layer_size = parameters['second_hidden_layer_size']
        second_hidden_layer_activation = parameters['second_hidden_layer_activation']
        input_dropout_do = parameters['input_dropout']
        hidden_dropout_do = parameters['hidden_dropout']

        GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
            first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
            second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
        )
        if GLOBAL_VARS.NN_MODEL_NAME in param_results_dict.keys() or GLOBAL_VARS.NN_MODEL_NAME in to_skip:
            print "skipping: {}".format(GLOBAL_VARS.NN_MODEL_NAME)
            continue
#         if first_hidden_layer_size < DOC2VEC_SIZE or second_hidden_layer_size < NN_OUTPUT_NEURONS:
#             print "skipping: {} due to 1st layer size {} < {} or 2nd layer size {} < {}".format(GLOBAL_VARS.NN_MODEL_NAME,
#                                                                                                 first_hidden_layer_size, DOC2VEC_SIZE, 
#                                                                                                 second_hidden_layer_size, NN_OUTPUT_NEURONS)
#             continue
            

        info('***************************************************************************************')
        info(GLOBAL_VARS.NN_MODEL_NAME)

        model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                                      first_hidden_layer_size, first_hidden_layer_activation, 
                                      second_hidden_layer_size, second_hidden_layer_activation, 
                                      input_dropout_do, hidden_dropout_do)
        model.summary()

        early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=EARLY_STOPPER_MIN_DELTA, \
                                                      patience=EARLY_STOPPER_PATIENCE, verbose=1, mode='auto')
        metrics_callback = MetricsCallback()

        # Model Fitting
        %time history = model.fit(x=X, y=y, validation_data=(Xv,yv), batch_size=NN_BATCH_SIZE, \
                                  nb_epoch=NN_MAX_EPOCHS, verbose=MODEL_VERBOSITY, callbacks=[early_stopper, metrics_callback])

        
        # using the recorded weights of the best recorded validation loss
        last_model_weights = model.get_weights()
        info('Evaluating on Validation Data using saved best weights')
        model.set_weights(metrics_callback.best_weights)
        yvp = model.predict(Xv)
        yvp_binary = get_binary_0_5(yvp)
        #print yvp
        info('Generating Validation Metrics')
        validation_metrics = get_metrics(yv, yvp, yvp_binary)
        print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
            validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
            validation_metrics['f1_micro'], validation_metrics['f1_macro'])
        best_validation_metrics = validation_metrics

        param_results_dict[GLOBAL_VARS.NN_MODEL_NAME] = dict()
        param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_validation_metrics'] = best_validation_metrics
        param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['epochs'] = len(history.history['val_loss'])
        param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_weights'] = metrics_callback.best_weights
#         param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_weights'] = last_model_weights

        duration = time.time() - start_time
        param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['duration'] =  duration

        del history, last_model_weights, metrics_callback, model

    del doc2vec_model
    
    if save_results:
        pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                                                       NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

2017-02-20 13:05:53,788 : INFO : loading Doc2Vec object from /mnt/data2/shalaby/parameter_search_doc2vec_models_new/full/doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None/epoch_8/model


********* doc2vec_size_200_w_2_type_dm_concat_0_mean_1_trainwords_0_hs_0_neg_10_vocabsize_None->8


In [103]:
%%time
pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

CPU times: user 1.34 s, sys: 44 ms, total: 1.39 s
Wall time: 1.4 s


In [43]:
for key in param_results_dict.keys():
    print('========== NN: {}'.format(key))
    val = param_results_dict[key]
#     val_metrics = val['last_validation_metrics']
    val_metrics2 =  val['best_validation_metrics']
    
    print('Epochs => {}'.format(val['epochs']))
    print('Best Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics2['coverage_error'], 
                                                                                        val_metrics2['f1_micro'], val_metrics2['f1_macro'],
                                                                                        val_metrics2['top_3']))
#     print('Best Val Loss => {}'.format(val["metrics_callback"].best_val_loss))
#     print('Last Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics['coverage_error'], 
#                                                                                         val_metrics['f1_micro'], val_metrics['f1_macro'],
#                                                                                         val_metrics['top_3']))


========== NN: nn_1st-size_500_1st-act_sigmoid_2nd-size_None_2nd-act_sigmoid_in-drop_False_hid-drop_True
Epochs => 26
Best Val: Coverage Error => 3.1476 | F1 Micro => 0.5901 | F1 Macro => 0.1052 | Top 3 => 0.8330
========== NN: nn_1st-size_500_1st-act_sigmoid_2nd-size_2000_2nd-act_relu_in-drop_False_hid-drop_True
Epochs => 44
Best Val: Coverage Error => 2.9189 | F1 Micro => 0.6550 | F1 Macro => 0.1744 | Top 3 => 0.8492
========== NN: nn_1st-size_500_1st-act_sigmoid_2nd-size_1000_2nd-act_relu_in-drop_False_hid-drop_True
Epochs => 45
Best Val: Coverage Error => 2.9413 | F1 Micro => 0.6516 | F1 Macro => 0.1737 | Top 3 => 0.8484
========== NN: nn_1st-size_500_1st-act_relu_2nd-size_2000_2nd-act_relu_in-drop_False_hid-drop_True
Epochs => 41
Best Val: Coverage Error => 2.9182 | F1 Micro => 0.6500 | F1 Macro => 0.1717 | Top 3 => 0.8481
========== NN: nn_1st-size_100_1st-act_sigmoid_2nd-size_1000_2nd-act_sigmoid_in-drop_False_hid-drop_True
Epochs => 25
Best Val: Coverage Error => 3.5353 | F1 Mi

## Run network for specific configuration

In [51]:
NN_BATCH_SIZE = 4096
NN_OUTPUT_NEURONS = len(classifications)

In [ ]:
start_time = time.time()
first_hidden_layer_size = 500
first_hidden_layer_activation = 'tanh'
second_hidden_layer_size = 2000
second_hidden_layer_activation = 'sigmoid'
input_dropout_do = False
hidden_dropout_do = True

#     print "===================================================================================\n" + \
#           "========== 1st Layer Size: {}, 1st Layer Activation: {}, \n 2nd Layer Size: {}, 2nd Layer Activation: {}, \n" + \
#           "Input Dropout: {}, Hidden Dropout: {} \n" + \
#           "==========================".format(first_hidden_layer_size, first_hidden_layer_activation, 
#                                                 second_hidden_layer_size, second_hidden_layer_activation, 
#                                                 input_dropout_do, hidden_dropout_do)

GLOBAL_VARS.NN_MODEL_NAME = 'nn_1st-size_{}_1st-act_{}_2nd-size_{}_2nd-act_{}_in-drop_{}_hid-drop_{}'.format(
    first_hidden_layer_size, first_hidden_layer_activation, second_hidden_layer_size, 
    second_hidden_layer_activation, input_dropout_do, hidden_dropout_do
)
if GLOBAL_VARS.NN_MODEL_NAME in param_results_dict.keys():
    print "Should be skipping: {}".format(GLOBAL_VARS.NN_MODEL_NAME)

info('***************************************************************************************')
info(GLOBAL_VARS.NN_MODEL_NAME)

model = create_keras_nn_model(DOC2VEC_SIZE, NN_OUTPUT_NEURONS, 
                              first_hidden_layer_size, first_hidden_layer_activation, 
                              second_hidden_layer_size, second_hidden_layer_activation, 
                              input_dropout_do, hidden_dropout_do)
model.summary()

early_stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=EARLY_STOPPER_MIN_DELTA, \
                                              patience=EARLY_STOPPER_PATIENCE, verbose=1, mode='auto')
metrics_callback = MetricsCallback()

# Model Fitting
%time history = model.fit(x=X, y=y, validation_data=(Xv,yv), batch_size=NN_BATCH_SIZE, \
                          nb_epoch=NN_MAX_EPOCHS, verbose=0, callbacks=[early_stopper, metrics_callback])

#     info('Evaluating on Training Data')
#     yp = model.predict(X, batch_size=NN_BATCH_SIZE)
#     yp_binary = get_binary_0_5(yp)
#     #print yp
#     info('Generating Training Metrics')
#     training_metrics = get_metrics(y, yp, yp_binary)
#     print "** Training Metrics: Cov Err: {:.3f}, Avg Labels: {:.3f}, \n\t\t Top 1: {:.3f}, Top 3: {:.3f}, Top 5: {:.3f}, \n\t\t F1 Micro: {:.3f}, F1 Macro: {:.3f}, Total Pos: {:,d}".format(
#         training_metrics['coverage_error'], training_metrics['average_num_of_labels'], 
#         training_metrics['top_1'], training_metrics['top_3'], training_metrics['top_5'], 
#         training_metrics['f1_micro'],training_metrics['f1_macro'],  training_metrics['total_positive'])

info('Evaluating on Validation Data using last weights')
yvp = model.predict(Xv)
yvp_binary = get_binary_0_5(yvp)
#print yvp
info('Generating Validation Metrics')
validation_metrics = get_metrics(yv, yvp, yvp_binary)
print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
    validation_metrics['f1_micro'], validation_metrics['f1_macro'])
last_validation_metrics = validation_metrics

# using the recorded weights of the best recorded validation loss
last_model_weights = model.get_weights()
info('Evaluating on Validation Data using saved best weights')
model.set_weights(metrics_callback.best_weights)
yvp = model.predict(Xv)
yvp_binary = get_binary_0_5(yvp)
#print yvp
info('Generating Validation Metrics')
validation_metrics = get_metrics_detailed(yv, yvp, yvp_binary)
print "****** Validation Metrics: Cov Err: {:.3f} | Top 3: {:.3f} | Top 5: {:.3f} | F1 Micro: {:.3f} | F1 Macro: {:.3f}".format(
    validation_metrics['coverage_error'], validation_metrics['top_3'], validation_metrics['top_5'], 
    validation_metrics['f1_micro'], validation_metrics['f1_macro'])
best_validation_metrics = validation_metrics


param_results_dict[GLOBAL_VARS.NN_MODEL_NAME] = dict()
#     param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['training_metrics'] = training_metrics
param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_validation_metrics'] = last_validation_metrics
param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['best_validation_metrics'] = best_validation_metrics
param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['history'] = history
param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['metrics_callback'] = metrics_callback
param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['last_weights'] = last_model_weights

duration = time.time() - start_time
param_results_dict[GLOBAL_VARS.NN_MODEL_NAME]['duration'] =  duration

pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                       NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))

In [141]:
for key in param_results_dict.keys():
    print('========== NN: {}'.format(key))
    val = param_results_dict[key]
    val_metrics = val['last_validation_metrics']
    val_metrics2 =  val['best_validation_metrics']
    
    print('Epochs => {}'.format(len(val['history'].history['val_loss'])))
    print('Best Val Loss => {}'.format(val["metrics_callback"].best_val_loss))
    print('Last Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics['coverage_error'], 
                                                                                        val_metrics['f1_micro'], val_metrics['f1_macro'],
                                                                                        val_metrics['top_3']))
    print('Best Val: Coverage Error => {:.4f} | F1 Micro => {:.4f} | F1 Macro => {:.4f} | Top 3 => {:.4f}'.format(val_metrics2['coverage_error'], 
                                                                                        val_metrics2['f1_micro'], val_metrics2['f1_macro'],
                                                                                        val_metrics2['top_3']))

========== NN: nn_1st-size_500_1st-act_softmax_2nd-size_None_2nd-act_softmax_in-drop_True_hid-drop_False
Epochs => 100
Best Val Loss => 0.166829405505
Last Val: Coverage Error => 1.4960 | F1 Micro => 0.7434 | F1 Macro => 0.6293 | Top 3 => 0.9637
Best Val: Coverage Error => 1.4960 | F1 Micro => 0.7434 | F1 Macro => 0.6293 | Top 3 => 0.9637
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_relu_in-drop_False_hid-drop_True
Epochs => 47
Best Val Loss => 0.147426413828
Last Val: Coverage Error => 1.4576 | F1 Micro => 0.7738 | F1 Macro => 0.7021 | Top 3 => 0.9697
Best Val: Coverage Error => 1.4576 | F1 Micro => 0.7738 | F1 Macro => 0.7021 | Top 3 => 0.9697
========== NN: nn_1st-size_200_1st-act_tanh_2nd-size_500_2nd-act_tanh_in-drop_True_hid-drop_True
Epochs => 28
Best Val Loss => 0.192695072037
Last Val: Coverage Error => 1.6299 | F1 Micro => 0.6631 | F1 Macro => 0.4731 | Top 3 => 0.9455
Best Val: Coverage Error => 1.6299 | F1 Micro => 0.6631 | F1 Macro => 0.4731 | Top 3 => 0

In [163]:
pickle.dump(param_results_dict, open(os.path.join(os.path.join(nn_parameter_search_location, GLOBAL_VARS.MODEL_NAME, 
                                           NN_PARAMETER_SEARCH_PREFIX.format(classifications_type, NN_BATCH_SIZE))), 'w'))